In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import multiprocessing
from multiprocessing.pool import ThreadPool

import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt
import tqdm
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [3]:
from data_io import load_data_by_ticker

In [11]:
calls_raw = pd.read_pickle('data/CALLS_CLIMATE.pkl')
calls = pd.read_pickle('data/CALLS_TRANSFORMED.pkl')

In [5]:
kospi_downloaded = pd.read_pickle('history/kospi_download_result.pkl')
kospi = pd.read_csv('metadata/kospi.csv', error_bad_lines=False)

b'Skipping line 437: expected 12 fields, saw 13\n'


# chicken calls

In [6]:
calls

,날짜,시군구,성별,연령대,총건수,월,요일,평균기온,최고기온,최저기온,...,최다풍향,평균상대습도,가조시간,일조시간,강설량,평균운량,미세먼지,공휴일_당일,공휴일_전날,일
0,2013-08-01,강남구,남,10대,71,8,목요일,28.0,24.5,32.1,...,50.0,67.3,14.1,6.0,0.0,7.1,34,0,0,1
1,2013-08-01,강남구,남,20대,241,8,목요일,28.0,24.5,32.1,...,50.0,67.3,14.1,6.0,0.0,7.1,34,0,0,1
2,2013-08-01,강남구,남,30대,276,8,목요일,28.0,24.5,32.1,...,50.0,67.3,14.1,6.0,0.0,7.1,34,0,0,1
3,2013-08-01,강남구,남,40대,142,8,목요일,28.0,24.5,32.1,...,50.0,67.3,14.1,6.0,0.0,7.1,34,0,0,1
4,2013-08-01,강남구,남,50대,44,8,목요일,28.0,24.5,32.1,...,50.0,67.3,14.1,6.0,0.0,7.1,34,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661436,2019-08-31,중랑구,여,20대,20,8,토요일,23.5,18.5,28.9,...,230.0,65.0,13.1,12.1,0.0,2.3,18,0,0,31
661437,2019-08-31,중랑구,여,30대,40,8,토요일,23.5,18.5,28.9,...,230.0,65.0,13.1,12.1,0.0,2.3,18,0,0,31
661438,2019-08-31,중랑구,여,40대,69,8,토요일,23.5,18.5,28.9,...,230.0,65.0,13.1,12.1,0.0,2.3,18,0,0,31
661439,2019-08-31,중랑구,여,50대,24,8,토요일,23.5,18.5,28.9,...,230.0,65.0,13.1,12.1,0.0,2.3,18,0,0,31


In [55]:
df_transformed = pd.DataFrame(index=calls_raw['날짜'].unique(),
                              columns=['월', '요일', '평균기온', '최고기온', '최저기온',
       '일강수량', '평균풍속', '최다풍향', '평균상대습도', '가조시간', '일조시간', '강설량', '평균운량', '미세먼지',
       '공휴일_당일', '공휴일_전날', '일',
       '총건수', '강남구, 10대, 남', '강남구, 10대, 여', '강남구, 20대, 남', '강남구, 20대, 여', '강남구, 30대, 남', '강남구, 30대, 여', '강남구, 40대, 남', '강남구, 40대, 여', '강남구, 50대, 남', '강남구, 50대, 여', '강남구, 60대, 남', '강남구, 60대, 여', '강동구, 10대, 남', '강동구, 10대, 여', '강동구, 20대, 남', '강동구, 20대, 여', '강동구, 30대, 남', '강동구, 30대, 여', '강동구, 40대, 남', '강동구, 40대, 여', '강동구, 50대, 남', '강동구, 50대, 여', '강동구, 60대, 남', '강동구, 60대, 여', '강북구, 10대, 남', '강북구, 10대, 여', '강북구, 20대, 남', '강북구, 20대, 여', '강북구, 30대, 남', '강북구, 30대, 여', '강북구, 40대, 남', '강북구, 40대, 여', '강북구, 50대, 남', '강북구, 50대, 여', '강북구, 60대, 남', '강북구, 60대, 여', '강서구, 10대, 남', '강서구, 10대, 여', '강서구, 20대, 남', '강서구, 20대, 여', '강서구, 30대, 남', '강서구, 30대, 여', '강서구, 40대, 남', '강서구, 40대, 여', '강서구, 50대, 남', '강서구, 50대, 여', '강서구, 60대, 남', '강서구, 60대, 여', '관악구, 10대, 남', '관악구, 10대, 여', '관악구, 20대, 남', '관악구, 20대, 여', '관악구, 30대, 남', '관악구, 30대, 여', '관악구, 40대, 남', '관악구, 40대, 여', '관악구, 50대, 남', '관악구, 50대, 여', '관악구, 60대, 남', '관악구, 60대, 여', '광진구, 10대, 남', '광진구, 10대, 여', '광진구, 20대, 남', '광진구, 20대, 여', '광진구, 30대, 남', '광진구, 30대, 여', '광진구, 40대, 남', '광진구, 40대, 여', '광진구, 50대, 남', '광진구, 50대, 여', '광진구, 60대, 남', '광진구, 60대, 여', '구로구, 10대, 남', '구로구, 10대, 여', '구로구, 20대, 남', '구로구, 20대, 여', '구로구, 30대, 남', '구로구, 30대, 여', '구로구, 40대, 남', '구로구, 40대, 여', '구로구, 50대, 남', '구로구, 50대, 여', '구로구, 60대, 남', '구로구, 60대, 여', '금천구, 10대, 남', '금천구, 10대, 여', '금천구, 20대, 남', '금천구, 20대, 여', '금천구, 30대, 남', '금천구, 30대, 여', '금천구, 40대, 남', '금천구, 40대, 여', '금천구, 50대, 남', '금천구, 50대, 여', '금천구, 60대, 남', '금천구, 60대, 여', '노원구, 10대, 남', '노원구, 10대, 여', '노원구, 20대, 남', '노원구, 20대, 여', '노원구, 30대, 남', '노원구, 30대, 여', '노원구, 40대, 남', '노원구, 40대, 여', '노원구, 50대, 남', '노원구, 50대, 여', '노원구, 60대, 남', '노원구, 60대, 여', '도봉구, 10대, 남', '도봉구, 10대, 여', '도봉구, 20대, 남', '도봉구, 20대, 여', '도봉구, 30대, 남', '도봉구, 30대, 여', '도봉구, 40대, 남', '도봉구, 40대, 여', '도봉구, 50대, 남', '도봉구, 50대, 여', '도봉구, 60대, 남', '도봉구, 60대, 여', '동대문구, 10대, 남', '동대문구, 10대, 여', '동대문구, 20대, 남', '동대문구, 20대, 여', '동대문구, 30대, 남', '동대문구, 30대, 여', '동대문구, 40대, 남', '동대문구, 40대, 여', '동대문구, 50대, 남', '동대문구, 50대, 여', '동대문구, 60대, 남', '동대문구, 60대, 여', '동작구, 10대, 남', '동작구, 10대, 여', '동작구, 20대, 남', '동작구, 20대, 여', '동작구, 30대, 남', '동작구, 30대, 여', '동작구, 40대, 남', '동작구, 40대, 여', '동작구, 50대, 남', '동작구, 50대, 여', '동작구, 60대, 남', '동작구, 60대, 여', '마포구, 10대, 남', '마포구, 10대, 여', '마포구, 20대, 남', '마포구, 20대, 여', '마포구, 30대, 남', '마포구, 30대, 여', '마포구, 40대, 남', '마포구, 40대, 여', '마포구, 50대, 남', '마포구, 50대, 여', '마포구, 60대, 남', '마포구, 60대, 여', '서대문구, 10대, 남', '서대문구, 10대, 여', '서대문구, 20대, 남', '서대문구, 20대, 여', '서대문구, 30대, 남', '서대문구, 30대, 여', '서대문구, 40대, 남', '서대문구, 40대, 여', '서대문구, 50대, 남', '서대문구, 50대, 여', '서대문구, 60대, 남', '서대문구, 60대, 여', '서초구, 10대, 남', '서초구, 10대, 여', '서초구, 20대, 남', '서초구, 20대, 여', '서초구, 30대, 남', '서초구, 30대, 여', '서초구, 40대, 남', '서초구, 40대, 여', '서초구, 50대, 남', '서초구, 50대, 여', '서초구, 60대, 남', '서초구, 60대, 여', '성동구, 10대, 남', '성동구, 10대, 여', '성동구, 20대, 남', '성동구, 20대, 여', '성동구, 30대, 남', '성동구, 30대, 여', '성동구, 40대, 남', '성동구, 40대, 여', '성동구, 50대, 남', '성동구, 50대, 여', '성동구, 60대, 남', '성동구, 60대, 여', '성북구, 10대, 남', '성북구, 10대, 여', '성북구, 20대, 남', '성북구, 20대, 여', '성북구, 30대, 남', '성북구, 30대, 여', '성북구, 40대, 남', '성북구, 40대, 여', '성북구, 50대, 남', '성북구, 50대, 여', '성북구, 60대, 남', '성북구, 60대, 여', '송파구, 10대, 남', '송파구, 10대, 여', '송파구, 20대, 남', '송파구, 20대, 여', '송파구, 30대, 남', '송파구, 30대, 여', '송파구, 40대, 남', '송파구, 40대, 여', '송파구, 50대, 남', '송파구, 50대, 여', '송파구, 60대, 남', '송파구, 60대, 여', '양천구, 10대, 남', '양천구, 10대, 여', '양천구, 20대, 남', '양천구, 20대, 여', '양천구, 30대, 남', '양천구, 30대, 여', '양천구, 40대, 남', '양천구, 40대, 여', '양천구, 50대, 남', '양천구, 50대, 여', '양천구, 60대, 남', '양천구, 60대, 여', '영등포구, 10대, 남', '영등포구, 10대, 여', '영등포구, 20대, 남', '영등포구, 20대, 여', '영등포구, 30대, 남', '영등포구, 30대, 여', '영등포구, 40대, 남', '영등포구, 40대, 여', '영등포구, 50대, 남', '영등포구, 50대, 여', '영등포구, 60대, 남', '영등포구, 60대, 여', '용산구, 10대, 남', '용산구, 10대, 여', '용산구, 20대, 남', '용산구, 20대, 여', '용산구, 30대, 남', '용산구, 30대, 여', '용산구, 40대, 남', '용산구, 40대, 여', '용산구, 50대, 남', '용산구, 50대, 여', '용산구, 60대, 남', '용산구, 60대, 여', '은평구, 10대, 남', '은평구, 10대, 여', '은평구, 20대, 남', '은평구, 20대, 여', '은평구, 30대, 남', '은평구, 30대, 여', '은평구, 40대, 남', '은평구, 40대, 여', '은평구, 50대, 남', '은평구, 50대, 여', '은평구, 60대, 남', '은평구, 60대, 여', '종로구, 10대, 남', '종로구, 10대, 여', '종로구, 20대, 남', '종로구, 20대, 여', '종로구, 30대, 남', '종로구, 30대, 여', '종로구, 40대, 남', '종로구, 40대, 여', '종로구, 50대, 남', '종로구, 50대, 여', '종로구, 60대, 남', '종로구, 60대, 여', '중구, 10대, 남', '중구, 10대, 여', '중구, 20대, 남', '중구, 20대, 여', '중구, 30대, 남', '중구, 30대, 여', '중구, 40대, 남', '중구, 40대, 여', '중구, 50대, 남', '중구, 50대, 여', '중구, 60대, 남', '중구, 60대, 여', '중랑구, 10대, 남', '중랑구, 10대, 여', '중랑구, 20대, 남', '중랑구, 20대, 여', '중랑구, 30대, 남', '중랑구, 30대, 여', '중랑구, 40대, 남', '중랑구, 40대, 여', '중랑구, 50대, 남', '중랑구, 50대, 여', '중랑구, 60대, 남', '중랑구, 60대, 여'])

calls_others = ['월', '요일', '평균기온', '최고기온', '최저기온',
       '일강수량', '평균풍속', '최다풍향', '평균상대습도', '가조시간', '일조시간', '강설량', '평균운량', '미세먼지',
       '공휴일_당일', '공휴일_전날', '일']
calls_by_sectors = ['강남구, 10대, 남', '강남구, 10대, 여', '강남구, 20대, 남', '강남구, 20대, 여', '강남구, 30대, 남', '강남구, 30대, 여', '강남구, 40대, 남', '강남구, 40대, 여', '강남구, 50대, 남', '강남구, 50대, 여', '강남구, 60대, 남', '강남구, 60대, 여', '강동구, 10대, 남', '강동구, 10대, 여', '강동구, 20대, 남', '강동구, 20대, 여', '강동구, 30대, 남', '강동구, 30대, 여', '강동구, 40대, 남', '강동구, 40대, 여', '강동구, 50대, 남', '강동구, 50대, 여', '강동구, 60대, 남', '강동구, 60대, 여', '강북구, 10대, 남', '강북구, 10대, 여', '강북구, 20대, 남', '강북구, 20대, 여', '강북구, 30대, 남', '강북구, 30대, 여', '강북구, 40대, 남', '강북구, 40대, 여', '강북구, 50대, 남', '강북구, 50대, 여', '강북구, 60대, 남', '강북구, 60대, 여', '강서구, 10대, 남', '강서구, 10대, 여', '강서구, 20대, 남', '강서구, 20대, 여', '강서구, 30대, 남', '강서구, 30대, 여', '강서구, 40대, 남', '강서구, 40대, 여', '강서구, 50대, 남', '강서구, 50대, 여', '강서구, 60대, 남', '강서구, 60대, 여', '관악구, 10대, 남', '관악구, 10대, 여', '관악구, 20대, 남', '관악구, 20대, 여', '관악구, 30대, 남', '관악구, 30대, 여', '관악구, 40대, 남', '관악구, 40대, 여', '관악구, 50대, 남', '관악구, 50대, 여', '관악구, 60대, 남', '관악구, 60대, 여', '광진구, 10대, 남', '광진구, 10대, 여', '광진구, 20대, 남', '광진구, 20대, 여', '광진구, 30대, 남', '광진구, 30대, 여', '광진구, 40대, 남', '광진구, 40대, 여', '광진구, 50대, 남', '광진구, 50대, 여', '광진구, 60대, 남', '광진구, 60대, 여', '구로구, 10대, 남', '구로구, 10대, 여', '구로구, 20대, 남', '구로구, 20대, 여', '구로구, 30대, 남', '구로구, 30대, 여', '구로구, 40대, 남', '구로구, 40대, 여', '구로구, 50대, 남', '구로구, 50대, 여', '구로구, 60대, 남', '구로구, 60대, 여', '금천구, 10대, 남', '금천구, 10대, 여', '금천구, 20대, 남', '금천구, 20대, 여', '금천구, 30대, 남', '금천구, 30대, 여', '금천구, 40대, 남', '금천구, 40대, 여', '금천구, 50대, 남', '금천구, 50대, 여', '금천구, 60대, 남', '금천구, 60대, 여', '노원구, 10대, 남', '노원구, 10대, 여', '노원구, 20대, 남', '노원구, 20대, 여', '노원구, 30대, 남', '노원구, 30대, 여', '노원구, 40대, 남', '노원구, 40대, 여', '노원구, 50대, 남', '노원구, 50대, 여', '노원구, 60대, 남', '노원구, 60대, 여', '도봉구, 10대, 남', '도봉구, 10대, 여', '도봉구, 20대, 남', '도봉구, 20대, 여', '도봉구, 30대, 남', '도봉구, 30대, 여', '도봉구, 40대, 남', '도봉구, 40대, 여', '도봉구, 50대, 남', '도봉구, 50대, 여', '도봉구, 60대, 남', '도봉구, 60대, 여', '동대문구, 10대, 남', '동대문구, 10대, 여', '동대문구, 20대, 남', '동대문구, 20대, 여', '동대문구, 30대, 남', '동대문구, 30대, 여', '동대문구, 40대, 남', '동대문구, 40대, 여', '동대문구, 50대, 남', '동대문구, 50대, 여', '동대문구, 60대, 남', '동대문구, 60대, 여', '동작구, 10대, 남', '동작구, 10대, 여', '동작구, 20대, 남', '동작구, 20대, 여', '동작구, 30대, 남', '동작구, 30대, 여', '동작구, 40대, 남', '동작구, 40대, 여', '동작구, 50대, 남', '동작구, 50대, 여', '동작구, 60대, 남', '동작구, 60대, 여', '마포구, 10대, 남', '마포구, 10대, 여', '마포구, 20대, 남', '마포구, 20대, 여', '마포구, 30대, 남', '마포구, 30대, 여', '마포구, 40대, 남', '마포구, 40대, 여', '마포구, 50대, 남', '마포구, 50대, 여', '마포구, 60대, 남', '마포구, 60대, 여', '서대문구, 10대, 남', '서대문구, 10대, 여', '서대문구, 20대, 남', '서대문구, 20대, 여', '서대문구, 30대, 남', '서대문구, 30대, 여', '서대문구, 40대, 남', '서대문구, 40대, 여', '서대문구, 50대, 남', '서대문구, 50대, 여', '서대문구, 60대, 남', '서대문구, 60대, 여', '서초구, 10대, 남', '서초구, 10대, 여', '서초구, 20대, 남', '서초구, 20대, 여', '서초구, 30대, 남', '서초구, 30대, 여', '서초구, 40대, 남', '서초구, 40대, 여', '서초구, 50대, 남', '서초구, 50대, 여', '서초구, 60대, 남', '서초구, 60대, 여', '성동구, 10대, 남', '성동구, 10대, 여', '성동구, 20대, 남', '성동구, 20대, 여', '성동구, 30대, 남', '성동구, 30대, 여', '성동구, 40대, 남', '성동구, 40대, 여', '성동구, 50대, 남', '성동구, 50대, 여', '성동구, 60대, 남', '성동구, 60대, 여', '성북구, 10대, 남', '성북구, 10대, 여', '성북구, 20대, 남', '성북구, 20대, 여', '성북구, 30대, 남', '성북구, 30대, 여', '성북구, 40대, 남', '성북구, 40대, 여', '성북구, 50대, 남', '성북구, 50대, 여', '성북구, 60대, 남', '성북구, 60대, 여', '송파구, 10대, 남', '송파구, 10대, 여', '송파구, 20대, 남', '송파구, 20대, 여', '송파구, 30대, 남', '송파구, 30대, 여', '송파구, 40대, 남', '송파구, 40대, 여', '송파구, 50대, 남', '송파구, 50대, 여', '송파구, 60대, 남', '송파구, 60대, 여', '양천구, 10대, 남', '양천구, 10대, 여', '양천구, 20대, 남', '양천구, 20대, 여', '양천구, 30대, 남', '양천구, 30대, 여', '양천구, 40대, 남', '양천구, 40대, 여', '양천구, 50대, 남', '양천구, 50대, 여', '양천구, 60대, 남', '양천구, 60대, 여', '영등포구, 10대, 남', '영등포구, 10대, 여', '영등포구, 20대, 남', '영등포구, 20대, 여', '영등포구, 30대, 남', '영등포구, 30대, 여', '영등포구, 40대, 남', '영등포구, 40대, 여', '영등포구, 50대, 남', '영등포구, 50대, 여', '영등포구, 60대, 남', '영등포구, 60대, 여', '용산구, 10대, 남', '용산구, 10대, 여', '용산구, 20대, 남', '용산구, 20대, 여', '용산구, 30대, 남', '용산구, 30대, 여', '용산구, 40대, 남', '용산구, 40대, 여', '용산구, 50대, 남', '용산구, 50대, 여', '용산구, 60대, 남', '용산구, 60대, 여', '은평구, 10대, 남', '은평구, 10대, 여', '은평구, 20대, 남', '은평구, 20대, 여', '은평구, 30대, 남', '은평구, 30대, 여', '은평구, 40대, 남', '은평구, 40대, 여', '은평구, 50대, 남', '은평구, 50대, 여', '은평구, 60대, 남', '은평구, 60대, 여', '종로구, 10대, 남', '종로구, 10대, 여', '종로구, 20대, 남', '종로구, 20대, 여', '종로구, 30대, 남', '종로구, 30대, 여', '종로구, 40대, 남', '종로구, 40대, 여', '종로구, 50대, 남', '종로구, 50대, 여', '종로구, 60대, 남', '종로구, 60대, 여', '중구, 10대, 남', '중구, 10대, 여', '중구, 20대, 남', '중구, 20대, 여', '중구, 30대, 남', '중구, 30대, 여', '중구, 40대, 남', '중구, 40대, 여', '중구, 50대, 남', '중구, 50대, 여', '중구, 60대, 남', '중구, 60대, 여', '중랑구, 10대, 남', '중랑구, 10대, 여', '중랑구, 20대, 남', '중랑구, 20대, 여', '중랑구, 30대, 남', '중랑구, 30대, 여', '중랑구, 40대, 남', '중랑구, 40대, 여', '중랑구, 50대, 남', '중랑구, 50대, 여', '중랑구, 60대, 남', '중랑구, 60대, 여']


calls_amount = ['총건수', '강남구, 10대, 남', '강남구, 10대, 여', '강남구, 20대, 남', '강남구, 20대, 여', '강남구, 30대, 남', '강남구, 30대, 여', '강남구, 40대, 남', '강남구, 40대, 여', '강남구, 50대, 남', '강남구, 50대, 여', '강남구, 60대, 남', '강남구, 60대, 여', '강동구, 10대, 남', '강동구, 10대, 여', '강동구, 20대, 남', '강동구, 20대, 여', '강동구, 30대, 남', '강동구, 30대, 여', '강동구, 40대, 남', '강동구, 40대, 여', '강동구, 50대, 남', '강동구, 50대, 여', '강동구, 60대, 남', '강동구, 60대, 여', '강북구, 10대, 남', '강북구, 10대, 여', '강북구, 20대, 남', '강북구, 20대, 여', '강북구, 30대, 남', '강북구, 30대, 여', '강북구, 40대, 남', '강북구, 40대, 여', '강북구, 50대, 남', '강북구, 50대, 여', '강북구, 60대, 남', '강북구, 60대, 여', '강서구, 10대, 남', '강서구, 10대, 여', '강서구, 20대, 남', '강서구, 20대, 여', '강서구, 30대, 남', '강서구, 30대, 여', '강서구, 40대, 남', '강서구, 40대, 여', '강서구, 50대, 남', '강서구, 50대, 여', '강서구, 60대, 남', '강서구, 60대, 여', '관악구, 10대, 남', '관악구, 10대, 여', '관악구, 20대, 남', '관악구, 20대, 여', '관악구, 30대, 남', '관악구, 30대, 여', '관악구, 40대, 남', '관악구, 40대, 여', '관악구, 50대, 남', '관악구, 50대, 여', '관악구, 60대, 남', '관악구, 60대, 여', '광진구, 10대, 남', '광진구, 10대, 여', '광진구, 20대, 남', '광진구, 20대, 여', '광진구, 30대, 남', '광진구, 30대, 여', '광진구, 40대, 남', '광진구, 40대, 여', '광진구, 50대, 남', '광진구, 50대, 여', '광진구, 60대, 남', '광진구, 60대, 여', '구로구, 10대, 남', '구로구, 10대, 여', '구로구, 20대, 남', '구로구, 20대, 여', '구로구, 30대, 남', '구로구, 30대, 여', '구로구, 40대, 남', '구로구, 40대, 여', '구로구, 50대, 남', '구로구, 50대, 여', '구로구, 60대, 남', '구로구, 60대, 여', '금천구, 10대, 남', '금천구, 10대, 여', '금천구, 20대, 남', '금천구, 20대, 여', '금천구, 30대, 남', '금천구, 30대, 여', '금천구, 40대, 남', '금천구, 40대, 여', '금천구, 50대, 남', '금천구, 50대, 여', '금천구, 60대, 남', '금천구, 60대, 여', '노원구, 10대, 남', '노원구, 10대, 여', '노원구, 20대, 남', '노원구, 20대, 여', '노원구, 30대, 남', '노원구, 30대, 여', '노원구, 40대, 남', '노원구, 40대, 여', '노원구, 50대, 남', '노원구, 50대, 여', '노원구, 60대, 남', '노원구, 60대, 여', '도봉구, 10대, 남', '도봉구, 10대, 여', '도봉구, 20대, 남', '도봉구, 20대, 여', '도봉구, 30대, 남', '도봉구, 30대, 여', '도봉구, 40대, 남', '도봉구, 40대, 여', '도봉구, 50대, 남', '도봉구, 50대, 여', '도봉구, 60대, 남', '도봉구, 60대, 여', '동대문구, 10대, 남', '동대문구, 10대, 여', '동대문구, 20대, 남', '동대문구, 20대, 여', '동대문구, 30대, 남', '동대문구, 30대, 여', '동대문구, 40대, 남', '동대문구, 40대, 여', '동대문구, 50대, 남', '동대문구, 50대, 여', '동대문구, 60대, 남', '동대문구, 60대, 여', '동작구, 10대, 남', '동작구, 10대, 여', '동작구, 20대, 남', '동작구, 20대, 여', '동작구, 30대, 남', '동작구, 30대, 여', '동작구, 40대, 남', '동작구, 40대, 여', '동작구, 50대, 남', '동작구, 50대, 여', '동작구, 60대, 남', '동작구, 60대, 여', '마포구, 10대, 남', '마포구, 10대, 여', '마포구, 20대, 남', '마포구, 20대, 여', '마포구, 30대, 남', '마포구, 30대, 여', '마포구, 40대, 남', '마포구, 40대, 여', '마포구, 50대, 남', '마포구, 50대, 여', '마포구, 60대, 남', '마포구, 60대, 여', '서대문구, 10대, 남', '서대문구, 10대, 여', '서대문구, 20대, 남', '서대문구, 20대, 여', '서대문구, 30대, 남', '서대문구, 30대, 여', '서대문구, 40대, 남', '서대문구, 40대, 여', '서대문구, 50대, 남', '서대문구, 50대, 여', '서대문구, 60대, 남', '서대문구, 60대, 여', '서초구, 10대, 남', '서초구, 10대, 여', '서초구, 20대, 남', '서초구, 20대, 여', '서초구, 30대, 남', '서초구, 30대, 여', '서초구, 40대, 남', '서초구, 40대, 여', '서초구, 50대, 남', '서초구, 50대, 여', '서초구, 60대, 남', '서초구, 60대, 여', '성동구, 10대, 남', '성동구, 10대, 여', '성동구, 20대, 남', '성동구, 20대, 여', '성동구, 30대, 남', '성동구, 30대, 여', '성동구, 40대, 남', '성동구, 40대, 여', '성동구, 50대, 남', '성동구, 50대, 여', '성동구, 60대, 남', '성동구, 60대, 여', '성북구, 10대, 남', '성북구, 10대, 여', '성북구, 20대, 남', '성북구, 20대, 여', '성북구, 30대, 남', '성북구, 30대, 여', '성북구, 40대, 남', '성북구, 40대, 여', '성북구, 50대, 남', '성북구, 50대, 여', '성북구, 60대, 남', '성북구, 60대, 여', '송파구, 10대, 남', '송파구, 10대, 여', '송파구, 20대, 남', '송파구, 20대, 여', '송파구, 30대, 남', '송파구, 30대, 여', '송파구, 40대, 남', '송파구, 40대, 여', '송파구, 50대, 남', '송파구, 50대, 여', '송파구, 60대, 남', '송파구, 60대, 여', '양천구, 10대, 남', '양천구, 10대, 여', '양천구, 20대, 남', '양천구, 20대, 여', '양천구, 30대, 남', '양천구, 30대, 여', '양천구, 40대, 남', '양천구, 40대, 여', '양천구, 50대, 남', '양천구, 50대, 여', '양천구, 60대, 남', '양천구, 60대, 여', '영등포구, 10대, 남', '영등포구, 10대, 여', '영등포구, 20대, 남', '영등포구, 20대, 여', '영등포구, 30대, 남', '영등포구, 30대, 여', '영등포구, 40대, 남', '영등포구, 40대, 여', '영등포구, 50대, 남', '영등포구, 50대, 여', '영등포구, 60대, 남', '영등포구, 60대, 여', '용산구, 10대, 남', '용산구, 10대, 여', '용산구, 20대, 남', '용산구, 20대, 여', '용산구, 30대, 남', '용산구, 30대, 여', '용산구, 40대, 남', '용산구, 40대, 여', '용산구, 50대, 남', '용산구, 50대, 여', '용산구, 60대, 남', '용산구, 60대, 여', '은평구, 10대, 남', '은평구, 10대, 여', '은평구, 20대, 남', '은평구, 20대, 여', '은평구, 30대, 남', '은평구, 30대, 여', '은평구, 40대, 남', '은평구, 40대, 여', '은평구, 50대, 남', '은평구, 50대, 여', '은평구, 60대, 남', '은평구, 60대, 여', '종로구, 10대, 남', '종로구, 10대, 여', '종로구, 20대, 남', '종로구, 20대, 여', '종로구, 30대, 남', '종로구, 30대, 여', '종로구, 40대, 남', '종로구, 40대, 여', '종로구, 50대, 남', '종로구, 50대, 여', '종로구, 60대, 남', '종로구, 60대, 여', '중구, 10대, 남', '중구, 10대, 여', '중구, 20대, 남', '중구, 20대, 여', '중구, 30대, 남', '중구, 30대, 여', '중구, 40대, 남', '중구, 40대, 여', '중구, 50대, 남', '중구, 50대, 여', '중구, 60대, 남', '중구, 60대, 여', '중랑구, 10대, 남', '중랑구, 10대, 여', '중랑구, 20대, 남', '중랑구, 20대, 여', '중랑구, 30대, 남', '중랑구, 30대, 여', '중랑구, 40대, 남', '중랑구, 40대, 여', '중랑구, 50대, 남', '중랑구, 50대, 여', '중랑구, 60대, 남', '중랑구, 60대, 여']

In [8]:
def transform_dataframe(date) : 
    target = calls_raw[calls_raw['날짜']==date]

    for other in calls_others : 
        df_transformed.loc[date][other] = target[other].iloc[0]
    total_call = 0
    for item in calls_by_sectors : 
        sector, age, sex = item.replace(' ', '').split(',')
        try :
            amount = target[(target['시군구']==sector) & (target['연령대']==age) & (target['성별']==sex)]['총건수'].values[0]
            total_call = total_call + amount
        except Exception as e : 
            amount = np.nan
        df_transformed.loc[date][item] = amount
    df_transformed.loc[date]['총건수'] = total_call
    pbar.update(1)
    return
    
#date = datetime.datetime(2013, 8, 1)
#transform_dataframe(date)
#display(df_transformed)

## transform with multiprocessing

In [ ]:
pool = ThreadPool(24)
with tqdm.tqdm(total=calls_raw['날짜'].unique().shape[0]) as pbar:
    for date in calls_raw['날짜'].unique() :
        pool.apply_async(transform_dataframe, [date])
    pool.close()
    pool.join()

display(df_transformed)
df_transformed.to_pickle('data/CALLS_TRANSFORMED.pkl')

 18%|█▊        | 405/2222 [15:19<1:09:10,  2.28s/it]

## transform without multiprocessing

In [ ]:
with tqdm.tqdm(total=calls['날짜'].unique().shape[0]) as pbar:
    for date in calls['날짜'].unique() :
        transform_dataframe(date)
        
display(df_transformed)
df_transformed.to_pickle('data/CALLS_TRANSFORMED.pkl')

# kospi metadata

In [6]:
kospi[0:2]

,번호,종목코드,기업명,업종코드,업종,상장주식수(주),자본금(원),액면가(원),통화구분,대표전화,주소,총카운트
0,1,095570,AJ네트웍스,147603.0,산업용 기계 및 장비 임대업,"46,822,295","46,822,295,000","1,000",원(KRW),02-6363-9999,"서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)",789.0
1,2,068400,AJ렌터카,147601.0,운송장비 임대업,"22,146,300","11,073,150,000",500,원(KRW),1544-1600,서울특별시 구로구 서부샛길 822,789.0


In [7]:
kospi.shape

(792, 12)

In [8]:
kospi['업종'].value_counts()

기타 금융업             58
자동차 신품 부품 제조업      42
의약품 제조업            39
1차 철강 제조업          34
기타 화학제품 제조업        30
                   ..
기타 전기장비 제조업         1
비알코올음료 및 얼음 제조업     1
재 보험업               1
의료용 기기 제조업          1
그외 기타 운송장비 제조업      1
Name: 업종, Length: 127, dtype: int64

In [9]:
kospi['업종코드'].value_counts()

116409.0    58
33003.0     42
32102.0     39
32401.0     34
32004.0     30
            ..
33304.0      1
33302.0      1
33109.0      1
20702.0      1
31008.0      1
Name: 업종코드, Length: 127, dtype: int64

In [10]:
def get_company_sector(ticker) : 
    df = kospi[kospi['종목코드']==ticker]
    return df['업종'].values[0]

kospi_downloaded['업종'] = kospi_downloaded['종목코드'].apply(lambda x : get_company_sector(x))
kospi_downloaded.to_pickle('history/kospi_download_result.pkl')
kospi_downloaded

,종목코드,상장일,결과,기업명,업종
0,095570,2015-08-21,Naver,AJ네트웍스,산업용 기계 및 장비 임대업
1,068400,2012-07-27,Naver,AJ렌터카,운송장비 임대업
2,006840,1999-08-11,Naver,AK홀딩스,기타 금융업
3,027410,2014-05-19,Naver,BGF,기타 금융업
4,282330,2017-12-08,Naver,BGF리테일,종합 소매업
...,...,...,...,...,...
783,079980,2012-02-23,Naver,휴비스,화학섬유 제조업
784,005010,1973-06-29,Naver,휴스틸,1차 철강 제조업
785,069260,2002-10-07,Naver,휴켐스,기타 화학제품 제조업
786,000540,1974-12-05,Naver,흥국화재,보험업


# kospi downloaded

In [11]:
kospi_downloaded.shape

(788, 5)

In [12]:
kospi_downloaded.head()

,종목코드,상장일,결과,기업명,업종
0,095570,2015-08-21,Naver,AJ네트웍스,산업용 기계 및 장비 임대업
1,068400,2012-07-27,Naver,AJ렌터카,운송장비 임대업
2,006840,1999-08-11,Naver,AK홀딩스,기타 금융업
3,027410,2014-05-19,Naver,BGF,기타 금융업
4,282330,2017-12-08,Naver,BGF리테일,종합 소매업


In [13]:
def get_company_name(ticker) : 
    return kospi[kospi['종목코드']==ticker]['기업명'].values[0]

kospi_downloaded['기업명'] = kospi_downloaded['종목코드'].apply(lambda x : get_company_name(x))
kospi_downloaded.to_pickle('history/kospi_download_result.pkl')

# Show by ticker

In [14]:
tickers = kospi_downloaded['종목코드']
tickers

0      095570
1      068400
2      006840
3      027410
4      282330
        ...  
783    079980
784    005010
785    069260
786    000540
787    003280
Name: 종목코드, Length: 788, dtype: object

In [15]:
@interact
def show_by_ticker(ticker = tickers) : 
    metadata = kospi[kospi['종목코드'] == ticker]
    display(metadata)
    
    data = pd.read_pickle('data/kospi/%s.data'%ticker)
    data.index = pd.to_datetime(data.index)
    data['Close'] = pd.to_numeric(data['Close'])
    data['Volume'] = pd.to_numeric(data['Volume'])
    display(data.head())
    
    fig = plt.figure(figsize=(10, 6))
    
    
    ax1 = fig.add_subplot(111)
    lns1 = ax1.plot(data.index, data['Close'], label='Close')
    plt.xticks(rotation=90)
    ax2 = ax1.twinx()
    lns2 = ax2.plot(data.index, data['Volume'], color='orange', alpha=0.4, label='Volume')
    
    lns = lns1+lns2
    labs = [l.get_label() for l in lns]
    plt.legend(lns, labs, loc=0)
    plt.title('ticker : %s, %s ~ %s' % (ticker, data.index[0], data.index[-1]))
    
    plt.show()

interactive(children=(Dropdown(description='ticker', options=('095570', '068400', '006840', '027410', '282330'…

# Show by company name

In [16]:
company_names = kospi_downloaded['기업명']
company_names

0      AJ네트웍스
1       AJ렌터카
2       AK홀딩스
3         BGF
4      BGF리테일
        ...  
783       휴비스
784       휴스틸
785       휴켐스
786      흥국화재
787      흥아해운
Name: 기업명, Length: 788, dtype: object

In [17]:
@interact
def show_by_company_name(company_name = company_names) : 
    metadata = kospi[kospi['기업명'] == company_name]
    ticker = metadata['종목코드'].values[0]
    display(metadata)
    
    data = pd.read_pickle('data/kospi/%s.data'%ticker)
    data.index = pd.to_datetime(data.index)
    data['Close'] = pd.to_numeric(data['Close'])
    data['Volume'] = pd.to_numeric(data['Volume'])
    display(data.head())
    
    fig = plt.figure(figsize=(10, 6))
    
    
    ax1 = fig.add_subplot(111)
    lns1 = ax1.plot(data.index, data['Close'], label='Close')
    plt.xticks(rotation=90)
    ax2 = ax1.twinx()
    lns2 = ax2.plot(data.index, data['Volume'], color='orange', alpha=0.4, label='Volume')
    
    lns = lns1+lns2
    labs = [l.get_label() for l in lns]
    plt.legend(lns, labs, loc=0)
    plt.title('ticker : %s, %s ~ %s' % (ticker, data.index[0], data.index[-1]))
    
    plt.show()

interactive(children=(Dropdown(description='company_name', options=('AJ네트웍스', 'AJ렌터카', 'AK홀딩스', 'BGF', 'BGF리테일…

# Preprocess calldata_raw

In [45]:
calls_raw['날짜'] = pd.to_datetime(calls_raw['날짜'])
calls_raw['일'] = calls_raw['날짜'].dt.day

In [46]:
calls_raw.shape

(661441, 22)

In [47]:
calls_raw.columns

Index(['날짜', '시군구', '성별', '연령대', '총건수', '월', '요일', '평균기온', '최고기온', '최저기온',
       '일강수량', '평균풍속', '최다풍향', '평균상대습도', '가조시간', '일조시간', '강설량', '평균운량', '미세먼지',
       '공휴일_당일', '공휴일_전날', '일'],
      dtype='object')

In [48]:
calls_raw

,날짜,시군구,성별,연령대,총건수,월,요일,평균기온,최고기온,최저기온,...,최다풍향,평균상대습도,가조시간,일조시간,강설량,평균운량,미세먼지,공휴일_당일,공휴일_전날,일
0,2013-08-01,강남구,남,10대,71,8,목요일,28.0,24.5,32.1,...,50.0,67.3,14.1,6.0,0.0,7.1,34,0,0,1
1,2013-08-01,강남구,남,20대,241,8,목요일,28.0,24.5,32.1,...,50.0,67.3,14.1,6.0,0.0,7.1,34,0,0,1
2,2013-08-01,강남구,남,30대,276,8,목요일,28.0,24.5,32.1,...,50.0,67.3,14.1,6.0,0.0,7.1,34,0,0,1
3,2013-08-01,강남구,남,40대,142,8,목요일,28.0,24.5,32.1,...,50.0,67.3,14.1,6.0,0.0,7.1,34,0,0,1
4,2013-08-01,강남구,남,50대,44,8,목요일,28.0,24.5,32.1,...,50.0,67.3,14.1,6.0,0.0,7.1,34,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661436,2019-08-31,중랑구,여,20대,20,8,토요일,23.5,18.5,28.9,...,230.0,65.0,13.1,12.1,0.0,2.3,18,0,0,31
661437,2019-08-31,중랑구,여,30대,40,8,토요일,23.5,18.5,28.9,...,230.0,65.0,13.1,12.1,0.0,2.3,18,0,0,31
661438,2019-08-31,중랑구,여,40대,69,8,토요일,23.5,18.5,28.9,...,230.0,65.0,13.1,12.1,0.0,2.3,18,0,0,31
661439,2019-08-31,중랑구,여,50대,24,8,토요일,23.5,18.5,28.9,...,230.0,65.0,13.1,12.1,0.0,2.3,18,0,0,31


In [49]:
calls_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 661441 entries, 0 to 661440
Data columns (total 22 columns):
날짜        661441 non-null datetime64[ns]
시군구       661441 non-null object
성별        661441 non-null object
연령대       661441 non-null object
총건수       661441 non-null int64
월         661441 non-null int64
요일        661441 non-null object
평균기온      661441 non-null float64
최고기온      661441 non-null float64
최저기온      661143 non-null float64
일강수량      661441 non-null float64
평균풍속      660545 non-null float64
최다풍향      659952 non-null float64
평균상대습도    661441 non-null float64
가조시간      661441 non-null float64
일조시간      658474 non-null float64
강설량       661441 non-null float64
평균운량      661441 non-null float64
미세먼지      661441 non-null int64
공휴일_당일    661441 non-null int64
공휴일_전날    661441 non-null int64
일         661441 non-null int64
dtypes: datetime64[ns](1), float64(11), int64(6), object(4)
memory usage: 111.0+ MB


In [32]:
calls_raw.select_dtypes('object')

,날짜,시군구,성별,연령대,요일
0,2013-08-01,강남구,남,10대,목요일
1,2013-08-01,강남구,남,20대,목요일
2,2013-08-01,강남구,남,30대,목요일
3,2013-08-01,강남구,남,40대,목요일
4,2013-08-01,강남구,남,50대,목요일
...,...,...,...,...,...
661436,2019-08-31,중랑구,여,20대,토요일
661437,2019-08-31,중랑구,여,30대,토요일
661438,2019-08-31,중랑구,여,40대,토요일
661439,2019-08-31,중랑구,여,50대,토요일


In [36]:
calls_raw.select_dtypes('int64')

,총건수,월,미세먼지,공휴일_당일,공휴일_전날
0,71,8,34,0,0
1,241,8,34,0,0
2,276,8,34,0,0
3,142,8,34,0,0
4,44,8,34,0,0
...,...,...,...,...,...
661436,20,8,18,0,0
661437,40,8,18,0,0
661438,69,8,18,0,0
661439,24,8,18,0,0


In [38]:
calls_raw.select_dtypes('float64')

,평균기온,최고기온,최저기온,일강수량,평균풍속,최다풍향,평균상대습도,가조시간,일조시간,강설량,평균운량
0,28.0,24.5,32.1,0.0,1.7,50.0,67.3,14.1,6.0,0.0,7.1
1,28.0,24.5,32.1,0.0,1.7,50.0,67.3,14.1,6.0,0.0,7.1
2,28.0,24.5,32.1,0.0,1.7,50.0,67.3,14.1,6.0,0.0,7.1
3,28.0,24.5,32.1,0.0,1.7,50.0,67.3,14.1,6.0,0.0,7.1
4,28.0,24.5,32.1,0.0,1.7,50.0,67.3,14.1,6.0,0.0,7.1
...,...,...,...,...,...,...,...,...,...,...,...
661436,23.5,18.5,28.9,0.0,1.9,230.0,65.0,13.1,12.1,0.0,2.3
661437,23.5,18.5,28.9,0.0,1.9,230.0,65.0,13.1,12.1,0.0,2.3
661438,23.5,18.5,28.9,0.0,1.9,230.0,65.0,13.1,12.1,0.0,2.3
661439,23.5,18.5,28.9,0.0,1.9,230.0,65.0,13.1,12.1,0.0,2.3


In [50]:
calls_raw.to_pickle('data/CALLS_CLIMATE.pkl')

# Preprocess call data transformed

In [34]:
for column in calls.columns : 
    try : 
        calls[column] = pd.to_numeric(calls[column])
    except Exception as e :
        print('%s : %s' % (column, e))

요일 : Unable to parse string "목요일" at position 0


In [35]:
calls.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2222 entries, 2013-08-01 to 2019-08-31
Columns: 318 entries, 월 to 중랑구, 60대, 여
dtypes: float64(130), int64(187), object(1)
memory usage: 5.4+ MB


In [36]:
calls.to_pickle('data/CALLS_TRANSFORMED.pkl')

In [38]:
calls.select_dtypes('object')

,요일
2013-08-01,목요일
2013-08-02,금요일
2013-08-03,토요일
2013-08-04,일요일
2013-08-05,월요일
...,...
2019-08-27,화요일
2019-08-28,수요일
2019-08-29,목요일
2019-08-30,금요일


In [39]:
calls.select_dtypes('int64')

,월,미세먼지,공휴일_당일,공휴일_전날,일,총건수,"강남구, 10대, 남","강남구, 10대, 여","강남구, 20대, 남","강남구, 20대, 여",...,"중구, 40대, 남","중구, 40대, 여","중구, 50대, 남","중랑구, 20대, 남","중랑구, 20대, 여","중랑구, 30대, 남","중랑구, 30대, 여","중랑구, 40대, 남","중랑구, 40대, 여","중랑구, 50대, 여"
2013-08-01,8,34,0,0,1,21371,71,60,241,206,...,32,35,16,79,91,98,133,62,88,55
2013-08-02,8,33,0,0,2,26880,120,52,286,281,...,40,50,23,115,99,177,160,82,101,44
2013-08-03,8,19,0,0,3,32317,116,67,314,309,...,54,58,26,174,164,169,207,108,136,48
2013-08-04,8,39,0,0,4,31314,82,79,325,325,...,30,42,18,125,149,204,188,98,120,46
2013-08-05,8,46,0,0,5,19174,85,48,236,226,...,21,36,11,95,70,103,117,56,65,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-27,8,26,0,0,27,7739,29,27,64,38,...,44,45,32,28,18,24,20,25,44,8
2019-08-28,8,29,0,0,28,8155,26,20,58,43,...,40,82,32,12,13,27,31,20,39,21
2019-08-29,8,28,0,0,29,8998,30,25,71,59,...,37,78,30,10,15,13,15,24,37,18
2019-08-30,8,19,0,0,30,12080,32,15,64,75,...,97,86,40,22,14,34,40,37,51,25


In [40]:
calls.select_dtypes('float64')

,평균기온,최고기온,최저기온,일강수량,평균풍속,최다풍향,평균상대습도,가조시간,일조시간,강설량,...,"중구, 10대, 남","중구, 10대, 여","중구, 50대, 여","중구, 60대, 남","중구, 60대, 여","중랑구, 10대, 남","중랑구, 10대, 여","중랑구, 50대, 남","중랑구, 60대, 남","중랑구, 60대, 여"
2013-08-01,28.0,24.5,32.1,0.0,1.7,50.0,67.3,14.1,6.0,0.0,...,11.0,18.0,17.0,6.0,NaN,33.0,31.0,33.0,NaN,6.0
2013-08-02,27.4,25.2,29.3,7.5,2.0,50.0,80.4,14.1,0.9,0.0,...,19.0,17.0,23.0,14.0,9.0,65.0,32.0,33.0,10.0,18.0
2013-08-03,28.9,26.7,32.6,0.1,3.7,230.0,72.3,14.0,8.9,0.0,...,16.0,9.0,27.0,7.0,9.0,59.0,50.0,36.0,17.0,19.0
2013-08-04,28.3,26.4,31.3,4.5,2.4,250.0,76.4,14.0,4.7,0.0,...,11.0,7.0,18.0,9.0,7.0,44.0,33.0,37.0,16.0,14.0
2013-08-05,27.7,26.4,30.1,7.0,3.2,230.0,78.9,14.0,4.5,0.0,...,16.0,8.0,14.0,NaN,7.0,49.0,20.0,30.0,8.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-27,25.7,22.2,29.3,20.0,1.0,290.0,71.1,13.2,5.6,0.0,...,10.0,10.0,28.0,16.0,34.0,5.0,5.0,12.0,8.0,10.0
2019-08-28,26.1,23.6,30.2,0.0,1.9,270.0,66.2,13.2,NaN,0.0,...,10.0,10.0,35.0,18.0,30.0,5.0,10.0,17.0,10.0,5.0
2019-08-29,23.4,20.1,26.4,36.9,2.2,160.0,77.1,13.1,4.9,0.0,...,22.0,15.0,49.0,21.0,34.0,9.0,10.0,9.0,11.0,10.0
2019-08-30,22.6,19.5,27.0,0.2,2.4,200.0,66.3,13.1,8.8,0.0,...,10.0,10.0,48.0,19.0,26.0,21.0,13.0,14.0,10.0,8.0


# Check correlation between calldata and stock price

In [41]:
kospi_downloaded

,종목코드,상장일,결과,기업명,업종
0,095570,2015-08-21,Naver,AJ네트웍스,산업용 기계 및 장비 임대업
1,068400,2012-07-27,Naver,AJ렌터카,운송장비 임대업
2,006840,1999-08-11,Naver,AK홀딩스,기타 금융업
3,027410,2014-05-19,Naver,BGF,기타 금융업
4,282330,2017-12-08,Naver,BGF리테일,종합 소매업
...,...,...,...,...,...
783,079980,2012-02-23,Naver,휴비스,화학섬유 제조업
784,005010,1973-06-29,Naver,휴스틸,1차 철강 제조업
785,069260,2002-10-07,Naver,휴켐스,기타 화학제품 제조업
786,000540,1974-12-05,Naver,흥국화재,보험업


In [44]:
calls

,월,요일,평균기온,최고기온,최저기온,일강수량,평균풍속,최다풍향,평균상대습도,가조시간,...,"중랑구, 20대, 남","중랑구, 20대, 여","중랑구, 30대, 남","중랑구, 30대, 여","중랑구, 40대, 남","중랑구, 40대, 여","중랑구, 50대, 남","중랑구, 50대, 여","중랑구, 60대, 남","중랑구, 60대, 여"
2013-08-01,8,목요일,28.0,24.5,32.1,0.0,1.7,50.0,67.3,14.1,...,79,91,98,133,62,88,33.0,55,NaN,6.0
2013-08-02,8,금요일,27.4,25.2,29.3,7.5,2.0,50.0,80.4,14.1,...,115,99,177,160,82,101,33.0,44,10.0,18.0
2013-08-03,8,토요일,28.9,26.7,32.6,0.1,3.7,230.0,72.3,14.0,...,174,164,169,207,108,136,36.0,48,17.0,19.0
2013-08-04,8,일요일,28.3,26.4,31.3,4.5,2.4,250.0,76.4,14.0,...,125,149,204,188,98,120,37.0,46,16.0,14.0
2013-08-05,8,월요일,27.7,26.4,30.1,7.0,3.2,230.0,78.9,14.0,...,95,70,103,117,56,65,30.0,28,8.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-27,8,화요일,25.7,22.2,29.3,20.0,1.0,290.0,71.1,13.2,...,28,18,24,20,25,44,12.0,8,8.0,10.0
2019-08-28,8,수요일,26.1,23.6,30.2,0.0,1.9,270.0,66.2,13.2,...,12,13,27,31,20,39,17.0,21,10.0,5.0
2019-08-29,8,목요일,23.4,20.1,26.4,36.9,2.2,160.0,77.1,13.1,...,10,15,13,15,24,37,9.0,18,11.0,10.0
2019-08-30,8,금요일,22.6,19.5,27.0,0.2,2.4,200.0,66.3,13.1,...,22,14,34,40,37,51,14.0,25,10.0,8.0


In [42]:
tickers = kospi_downloaded['종목코드']

In [66]:
def get_call_data_for_given_ticker(calls, ticker) : 
    metadata, stock = load_data_by_ticker(kospi, ticker)
    
    # 1. stock
    #display(stock)
    
    # 2. calls
    chicken = calls[calls_amount]
    chicken.index = calls.index
    #display(chicken)
    
    # 3. merge to dataframe to get common period
    df_totest = stock.merge(chicken, how='inner', left_on=stock.index, right_on=chicken.index)
    df_totest.index = df_totest['key_0']
    df_totest = df_totest.drop(['key_0'], axis=1)
    
    return df_totest

In [67]:
def get_correlation_by_ticker(calls, ticker) : 
    
    df_totest = get_call_data_for_given_ticker(calls, ticker)
    display(df_totest.head())
    
get_correlation_by_ticker(calls, '079980')

,High,Low,Open,Close,Volume,Adj Close,총건수,"강남구, 10대, 남","강남구, 10대, 여","강남구, 20대, 남",...,"중랑구, 20대, 남","중랑구, 20대, 여","중랑구, 30대, 남","중랑구, 30대, 여","중랑구, 40대, 남","중랑구, 40대, 여","중랑구, 50대, 남","중랑구, 50대, 여","중랑구, 60대, 남","중랑구, 60대, 여"
key_0,,,,,,,,,,,,,,,,,,,,,
2013-08-01,11800,11400,11400,11700,240454,,21371,71,60,241,...,79,91,98,133,62,88,33.0,55,NaN,6.0
2013-08-02,11850,11400,11850,11600,236228,,26880,120,52,286,...,115,99,177,160,82,101,33.0,44,10.0,18.0
2013-08-05,11900,11450,11500,11900,159448,,19174,85,48,236,...,95,70,103,117,56,65,30.0,28,8.0,12.0
2013-08-06,11900,11550,11850,11600,96026,,20764,103,68,247,...,91,85,114,129,65,77,29.0,33,8.0,14.0
2013-08-07,11650,11150,11500,11250,152511,,21619,81,69,240,...,101,83,98,105,44,75,27.0,37,8.0,16.0
